# Day 24: Crossed Wires

https://adventofcode.com/2024/day/24

## --- Part One ---

In [54]:
# some imports i may or may not need 
import numpy as np
import math
import re
import heapq
from copy import deepcopy
from pprint import pprint
from termcolor import colored
import matplotlib.pyplot as plt

file = 'sample.txt'
file = 'sample2.txt'
file = 'input.txt'

# init vars
wires = {}
instructions = []

def convert_logic(logic):
    if logic == 'AND':
        return '&'
    elif logic == 'XOR':
        return '^'
    else:
        return '|'

# open file & load content
with open(file, 'r') as f:
    [a,b] = [line.splitlines() for line in f.read().split('\n\n')]
    # parse the initial wires with values
    for item in a:
        aa = re.match(r"^(.{3}): (\d)", item)
        wires[aa.group(1)] = int(aa.group(2))
    for item in b:
        aa = re.match(r"^(.{3}) (AND|XOR|OR) (.{3}) -> (.{3})", item)
        wire1 = aa.group(1)
        logic = aa.group(2)
        wire2 = aa.group(3)
        wire3 = aa.group(4)
        local_wires = [wire1,wire2,wire3]
        local_instructions = [wire1, convert_logic(logic), wire2, wire3]
        instructions.append(local_instructions)
        for wire in local_wires:
            if wire not in wires:
                wires[wire] = None


def z_done():
    for wire in wires:
        if wire[0] == 'z':
            if wires[wire] == None:
                return False
        
    return True

def calc_outcome():
    sorted_keys = sorted(wires.keys())
    local_result = ''
    for key in sorted_keys:
        if key[0]== 'z':
            local_result += str(wires[key])
    inv_outcome = local_result[::-1]
    return int(inv_outcome,2)

while not z_done():
    for instruction in instructions:
        [wire1, logic, wire2, wire3] = instruction
        if wires[wire1] != None and wires[wire2] != None:
            outcome = eval(f"{wires[wire1]} {logic} {wires[wire2]}")
            wires[wire3] = outcome

result = calc_outcome()
print('Answer to part 1:', result)

Answer to part 1: 57632654722854


## --- Part Two ---

In [123]:
# some imports i may or may not need 
import numpy as np
import math
import re
import heapq
from copy import deepcopy
from pprint import pprint
from termcolor import colored
import matplotlib.pyplot as plt
import random

file = 'sample.txt'
file = 'sample3.txt'
file = 'input.txt'

# init vars
wires = {}
instructions = []
checked = set(())

def convert_logic(logic):
    if logic == 'AND':
        return '&'
    elif logic == 'XOR':
        return '^'
    else:
        return '|'

# open file & load content
with open(file, 'r') as f:
    [a,b] = [line.splitlines() for line in f.read().split('\n\n')]
    # parse the initial wires with values
    for item in a:
        aa = re.match(r"^(.{3}): (\d)", item)
        wires[aa.group(1)] = int(aa.group(2))
    for item in b:
        aa = re.match(r"^(.{3}) (AND|XOR|OR) (.{3}) -> (.{3})", item)
        wire1 = aa.group(1)
        logic = aa.group(2)
        wire2 = aa.group(3)
        wire3 = aa.group(4)
        local_wires = [wire1,wire2,wire3]
        local_instructions = [wire1, convert_logic(logic), wire2, wire3]
        instructions.append(local_instructions)
        for wire in local_wires:
            if wire not in wires:
                wires[wire] = None


def z_done(xwires):
    for wire in xwires:
        if wire[0] == 'z':
            if xwires[wire] == None:
                return False
        
    return True

def calc_outcome(local_wires):
    sorted_keys = sorted(local_wires.keys())
    x = ''
    y = ''
    z = ''
    for key in sorted_keys:
        if key[0]== 'x':
            x += str(local_wires[key])
        if key[0]== 'y':
            y += str(local_wires[key])
        if key[0]== 'z':
            z += str(local_wires[key])
    inv_x_binary = x[::-1]
    inv_y_binary = y[::-1]
    inv_z_binary = z[::-1]
    # calculate the decimal value result
    # print('Trying', int(inv_x_binary, 2), '+',int(inv_y_binary,2),'=', int(inv_z_binary,2))
    if int(inv_x_binary, 2) + int(inv_y_binary,2) == int(inv_z_binary,2): # use for input data
    # if int(inv_x_binary, 2) & int(inv_y_binary,2) == int(inv_z_binary,2): # use for sample data
        return True
    return False


def generate_unique_random(x, excluded_list):
    while True:
        random_int = random.randint(0, x)
        if random_int not in excluded_list:
            return random_int

def generated_unique_x_random_nrs(x, max_nr):
    global checked
    while True:
        generated = []
        same = []
        for i in range(0,x):
            nr = generate_unique_random(max_nr, generated)
            generated.append(nr)
            if i%2 !=0 :
                same.insert(i-1, nr)
            else:
                same.insert(i, nr)
        if tuple(generated) not in checked:
            checked.add(tuple(generated))
            checked.add(tuple(same))
            return generated

done = False

while not done:
    # nrs = 4 # for sample data
    nrs = 8 # for input data
    new_wires = deepcopy(wires)
    new_instructions = deepcopy(instructions)
    gen4 = generated_unique_x_random_nrs(nrs, len(instructions)-1)
    # gen4 = [0,5,1,2]
    for i in range(nrs):
        if i%2 ==0 :
            new_instructions[gen4[i]][3] =  instructions[gen4[i+1]][3]
            new_instructions[gen4[i+1]][3] =  instructions[gen4[i]][3]
    # replace outcomes
    max_loops =30
    j = 0
    terminated = False
    while not z_done(new_wires):
    # while not calc_outcome():
        for instruction in new_instructions:
            [wire1, logic, wire2, wire3] = instruction
            if new_wires[wire1] != None and new_wires[wire2] != None:
                outcome = eval(f"{new_wires[wire1]} {logic} {new_wires[wire2]}")
                new_wires[wire3] = outcome

        j+=1
        if j>=max_loops:
            terminated = True
            break
    if not terminated:
        done = calc_outcome(new_wires)
        # print(gen4)
        if done:
            print(gen4)
        # print('---')

# result = calc_outcome(new_wires)
# print('Answer to part 1:', result)

KeyboardInterrupt: 